# Tidetools Module Functionalities

- In these examples, methods used to predict tide are adapated from Pytides
- This implementation will only work for known NOAA gauge stations
- Harmonic Constituents data is fetched from NOAA. 

Adapted by @rjleveque from notebook of @socoyjonathan to test local pytides and new tidetools.py.

## Tide Prediction Module Functions

 - <b>fetch_harcon</b> - Fetches harmonic constituents for CO-OPS station.
 - <b>make_pytides_model</b> - Fetches harmonic constituents for station
 - <code><b>fetch_noaa_tide_data()</b></code> - retrieves datetimes, water levels and tide predictions at given NOAA tide station from NOAA's API
 - <b>fetch_datums</b> - Fetch datums for CO-OPS station.
 - <code><b>predict_tide()</b></code> - predicts tide for desired NOAA station given station ID, start date and end date for prediction 
 - <b>datetimes</b> - prepares a collection of datetimes from beginning to end dates as needed
 - <code><b>detide()</b></code> - detides observed water levels with predicted tide
 - <code><b>surge()</b></code> - predicts surge at NOAA gauge station provided station ID, start date, end date, and landfall date! 

In [ ]:
from clawpack.geoclaw import tidetools
import matplotlib.pyplot as plt
import datetime

*******************************************************************************************************************

# Example of Tide Prediction For One Date Instance

### ****  Station Information ****

Locate NOAA station ID.  NOAA gauge stations home: https://tidesandcurrents.noaa.gov/ <br>
Fill in station ID, reference datum and date instance for tide prediction!

In [ ]:
#Station Information
station_id = '8761724'
datum = 'MTL'

#Date of prediction (YEAR, MTH, DAY, HR)
prediction_date = datetime.datetime(2005, 8, 29, 11)

### Tide Prediction

Prediction of tide at specified location (station ID) and specified time (GMT) implemented below by calling <code><b>predict_tide()</b></code> method with the following arguments: <b> station_id, beg_prediction_date, end_prediction_date</b>. Note: datum, time zone and units arguments are optional

<br> 

To predict tide at an instant, set <b>beg_prediction_date</b> and <b>end_prediction_date</b> in <code><b>predict_tide()</b></code> method to the same date!

In [ ]:
#NOAA Data Scraping Implementation      
height = tidetools.predict_tide(station_id, prediction_date, prediction_date, datum)
times = tidetools.datetimes(prediction_date, prediction_date) # in meters
print(height[0], "meters")

*******************************************************************************************************************

# Example of Tide Prediction In A Date Interval 

###   Station Information 

Fill in station ID, a beginning date and an end date for tide prediction below

In [ ]:
#Station Information
station_id = '8761724'
datum = 'MTL'

#Beginning and End Dates 
beg_date = datetime.datetime(2005, 8, 26, hour=0)
end_date = datetime.datetime(2005, 8, 31, hour=0)

#Predict tide with arguments set as: (station_id, beg_pred_date, end_pred_date)
predicted_tide = tidetools.predict_tide(station_id, beg_date, end_date, datum)

### Tide Predictions
Plot results in a time series plot

In [ ]:
#Method datetimes() makes a range of datetimes given args: (beg_pred_date, end_pred_date)
times = tidetools.datetimes(beg_date, end_date)

plt.figure(figsize=(20,10))
plt.plot(times, predicted_tide, "-", label="Tide Prediction")
plt.xlabel('Hours since ' + str(beg_date) + ' (GMT)')
plt.ylabel('Meters'), plt.margins(x=0), plt.legend(loc = 'best')
plt.title('Pytide Tide Prediction for Station {}'.format(station_id))
plt.show()

*******************************************************************************************************************

# Example Comparing Pytides vs NOAA Tide Prediction In A Date Interval 

In this example, we implement the Pytides submodule to predict the tide from <b>Hurricane Katrina</b> in 2005 on Grand Isle, LA (NOAA gauge station: 8761724) between start date, 08-26-2005, and end date, 08-31-2005. NOAA's predicted tide and observed water levels are also displayed for comparison.

In [ ]:
#Station Information
station_id = '8761724'
datum = 'MTL'

#Beginning and End Dates 
beg_date = datetime.datetime(2005, 8, 26)
end_date = datetime.datetime(2005, 8, 31)

#Predict Tide 
predicted_tide = tidetools.predict_tide(station_id, beg_date, end_date, datum)

- Calling function <code><b>fetch_noaa_tide_data()</b></code> with arguments set as <b>(station_id, beg_prediction_date, end_prediction_date)</b> retrieves datetimes, water levels and tide predictions for the specified NOAA station in the date interval provided from NOAA's API
- Data is fetched in <b>Metric</b> units, <b>GMT</b> timezone, <b>MTL</b> datum and  <b>6 min</b> intervals. These arguments are optional in <code><b>fetch_noaa_tide_data()</b></code>.

In [ ]:
#Retrieve NOAA Tide Data
times, NOAA_observed_water_lvl, NOAA_predicted_tide = \
        tidetools.fetch_noaa_tide_data(station_id, beg_date, end_date)

In [ ]:
#Plot Comparisons
plt.figure(figsize=(20,10))
plt.plot(times, predicted_tide, "-", label="Pytides Prediction")
plt.plot(times, NOAA_predicted_tide, "-", label="NOAA Tide Prediction")
plt.plot(times, NOAA_observed_water_lvl, "-", label="NOAA Water Level Observation")
plt.xlabel('Hours since ' + str(beg_date) + ' (GMT)')
plt.ylabel('Metres'), plt.margins(x=0), plt.legend(loc = 'best')
plt.title('Comparison of Pytides Tide Prediction vs NOAA Prediction for Station {}'.format(station_id))
plt.show()

*******************************************************************************************************************

# Example Detiding and Capturing A Surge for a Gauge Station 

In this example, we detide the observed water levels for <b>Hurricane Katrina</b> in 2005 on Grand Isle, LA (NOAA gauge station: 8761724) between start date, 08-26-2005, and end date, 08-31-2005, using Pytides predicted tide. With this method, one may compare the storm surge from Pytides prediction with GeoClaw simulation results as shown in the next example.

- Calling <code><b>predict_tide()</b></code> method with arguments set as: <b>(station_id, beg_prediction_date, end_prediction_date)</b> outputs predicted tide
- Calling <code><b>fetch_noaa_tide_data()</b></code> with arguments set as <b>(station_id, beg_prediction_date, end_prediction_date)</b> retrieves datetimes, water levels and tide predictions from NOAA
- Calling <code><b>detide()</b></code> method with arguments set as: <b>(NOAA observed water level, predicted tide)</b> will output detided water level. 

In [ ]:
#Station Information
station_id = '8761724'
datum = 'MTL'

#Beginning and End Dates 
beg_date = datetime.datetime(2005, 8, 26)
end_date = datetime.datetime(2005, 8, 31)

predicted_tide = tidetools.predict_tide(station_id, beg_date, end_date)
times, NOAA_observed_water_lvl, NOAA_predicted_tide = \
       tidetools.fetch_noaa_tide_data(station_id, beg_date, end_date, datum)

surge = tidetools.detide(NOAA_observed_water_lvl, predicted_tide)

#Plot Comparisons
plt.figure(figsize=(20,10))
plt.plot(times, surge, "-", label="Pytides Storm Surge Prediction")
plt.xlabel('Hours since ' + str(beg_date) + ' (GMT)')
plt.ylabel('Metres'), plt.margins(x=0), plt.legend(loc = 'best')
plt.title('Detided Water Level for Station {}'.format(station_id))
plt.show()

*******************************************************************************************************************

# Example Implementation of Pytides Tide Prediction on Clawpack

- Code below may be utilized to compare NOAA guage results with Clawpack storm surge simulations. 
- Code below works best if placed in <b>gauge_afteraxes( )</b> in <b>setplot.py</b> for comparison.
- Calling <code><b>surge()</b></code> method with arguments set as: <b>(station_id, beginning_date, end_date, landfall_date)</b> will output predicted storm surge from NOAA observed water levels minus Pytides submodule predicted tide.

In [ ]:
#Station Information
station_id = '8761724'

#Beginning, End, Landfall Dates
beg_date = datetime.datetime(2005, 8, 26)
end_date = datetime.datetime(2005, 8, 31)
landfall_date = datetime.datetime(2005, 8, 29, 11, 10)

# Surge Prediction
times, surge = tidetools.surge(station_id, beg_date, end_date, landfall_date)
plt.plot(times, surge, color="b", label="Pytides Prediction")

*******************************************************************************************************************

# Example Iterating Through A Library Of Stations And Date Intervals

In this example, we provide tide and storm surge predictions for five different storm events utilizing Pytides submodule and data from NOAA. 

1) First storm event considered is [Hurricane Katrina](https://www.nhc.noaa.gov/data/tcr/AL122005_Katrina.pdf) on NOAA station [Grand Isle, LA](https://tidesandcurrents.noaa.gov/stationhome.html?id=8761724) with a storm surge within the given dates. 

2) Second storm event considered is [Hurricane Michael](https://www.nhc.noaa.gov/data/tcr/AL142018_Michael.pdf) on NOAA station [Pilots Station East, SW Pass, LA](https://tidesandcurrents.noaa.gov/stationhome.html?id=8760922) with a storm surge within the given dates. 

3) Third storm event considered is [Hurricane Matthew](https://www.nhc.noaa.gov/data/tcr/AL142016_Matthew.pdf) on NOAA station [Wilmington, NC](https://tidesandcurrents.noaa.gov/stationhome.html?id=8658120) with a storm surge within the given dates. 

5) Fourth storm event considered is [Hurricane Irma](https://www.nhc.noaa.gov/data/tcr/AL112017_Irma.pdf) on NOAA station [Vaca Key, Florida Bay, FL](https://tidesandcurrents.noaa.gov/stationhome.html?id=8723970) with a storm surge within the given dates. 

4) Fifth storm event considered is [Hurricane Dorian](https://www.nhc.noaa.gov/data/tcr/AL052019_Dorian.pdf) on NOAA station [Trident Pier, Port Canaveral, FL](https://tidesandcurrents.noaa.gov/stationhome.html?id=8721604) with a storm surge within the given dates. 

In [ ]:
station_dict = {'8761724': ('Grand Isle, LA', (2005, 8, 26), (2005, 8, 31), (2005, 8, 29, 11, 10)), #katrina
                '8760922': ('Pilots Station East, SW Pass, LA', (2005, 8, 26), (2005, 8, 31), (2005, 8, 29, 11)), #michael
                '8658120': ('Wilmington, NC', (2016, 10, 6, 12), (2016, 10, 9, 12), (2016, 10, 8, 12)), #matthew
                '8723970': ('Vaca Key, Florida Bay, FL', (2017, 9, 6, 13), (2017, 9, 12, 13), (2017, 9, 10, 13)), #irma
                '8721604': ('Trident Pier, Port Canaveral, FL', (2019, 8, 24), (2019, 9, 9), (2019, 9, 4, 12)) #dorian
               }

for (key, value) in station_dict.items():
    station_id = key
    station_name = value[0]
    beg_date = datetime.datetime(*value[1])
    end_date = datetime.datetime(*value[2])
    landfall_date = datetime.datetime(*value[3])
    
    #NOAA Data Scraping Implementation
    predicted_tide = tidetools.predict_tide(station_id, beg_date, end_date) 
    
    times, NOAA_observed_water_lvl, NOAA_predicted_tide = \
           tidetools.fetch_noaa_tide_data(station_id, beg_date, end_date)

    #Detide Water Level
    surge = tidetools.detide(NOAA_observed_water_lvl, predicted_tide)
    NOAA_surge = tidetools.detide(NOAA_observed_water_lvl, NOAA_predicted_tide)
    
    #Plot Comparisons
    plt.figure(figsize=(20,10))
    plt.plot(times, predicted_tide, "-", label="Pytides Tide Prediction")
    plt.plot(times, NOAA_predicted_tide, "-", label="NOAA Tide Prediction")
    plt.plot(times, NOAA_observed_water_lvl, "-", label="NOAA Water Level Observation")
    plt.xlabel('Hours since ' + str(beg_date) + ' (GMT)')
    plt.ylabel('Metres'), plt.margins(x=0), plt.legend(loc = 'best')
    plt.title('Comparison of Pytides Prediction vs NOAA Prediction for Station {}, {}'\
              .format(station_id, station_name))
    plt.show()
    
    #Detided Water Level Comparison
    plt.figure(figsize=(20,10))
    plt.plot(times, surge, "-", label="Pytides Detided Prediction")
    plt.plot(times, NOAA_surge, "-", label="NOAA Detided Prediction")
    plt.xlabel('Hours since ' + str(beg_date) + ' (GMT)')
    plt.ylabel('Metres'), plt.margins(x=0), plt.legend(loc = 'best')
    plt.title('Detided Water Level Comparison of Pytides Prediction vs NOAA Prediction for Station {}, {}'\
              .format(station_id, station_name))
    plt.show()

From the example above, one may observe that the NOAA water level observations before the storm surge deviate significantly from both Pytides and NOAA's predicted tide levels. Many factors may contribute to this significant discrepency in observed water levels and predicted tide levels way before the storm surge event. 

This discrepency may be most likely due to the effects of the storm event already affecting the water levels days prior to the event in the vicinity of the specified location. These effects may include changes in waves and wind setup, ocean and river currents, temperature of the ocean water, barometric pressure, and relative sea level changes during the time interval leading up to the event as presented by [NOAA] (https://www.nhc.noaa.gov/surge/surge_intro.pdf).

However, other factor may contribute to this discrepency such as the difference in the actual tide level taking place and predicted tide level. Since a tide is the alternating rise and fall of the oceans with respect to the land, mainly produced by the gravitational attraction of the moon and sun but also due to non-astronomical factors such as the configuration of the coastline, local depth of the water and ocean-floor topography,  the range of the tides, the times of arrival of the tides, and the time interval between high and low water will deviate from the harmonic analysis utilized by Pytides and NOAA as presented by [Paul Schureman](https://tidesandcurrents.noaa.gov/publications/SpecialPubNo98.pdf).